# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель:** анализ базы данных о книгах, издательсвах, авторах и обзорах книг, для использования в сервисе для чтения книг.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

In [3]:
books = '''SELECT * FROM books LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [4]:
authors = '''SELECT * FROM authors LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt


In [5]:
publishers = '''SELECT * FROM publishers LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace


In [6]:
ratings = '''SELECT * FROM ratings LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [7]:
reviews = '''SELECT * FROM reviews LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## Анализ данных

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
task = '''SELECT COUNT(*) FROM books 
where CAST(publication_date as date) > '2000-01-01'
'''

con = engine.connect()
task = pd.io.sql.read_sql(sql = text(task), con = con)
print('Количество книг, вышедших после 1 января 2000 года:', int(task['count']) )

Количество книг, вышедших после 1 января 2000 года: 819


После 2000 года количество выпущенных книг составило 819 штук

## Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [9]:
task1 = '''SELECT title,review,rating
   FROM books as b
LEFT JOIN (SELECT book_id, count(review_id) as review
          FROM reviews
        GROUP BY  book_id) as re ON b.book_id=re.book_id
LEFT JOIN (SELECT book_id,  ROUND(AVG(rating),2) as rating
          FROM ratings
        GROUP BY book_id ) as ra ON b.book_id=ra.book_id
'''

con = engine.connect()
task = pd.io.sql.read_sql(sql = text(task1), con = con)
print("Оценка книг")
display(task)

Оценка книг


,title,review,rating
0,The Body in the Library (Miss Marple #3),2.0,4.50
1,Galápagos,2.0,4.50
2,A Tree Grows in Brooklyn,5.0,4.25
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,The Prophet,4.0,4.29
...,...,...,...
995,Alice in Wonderland,4.0,4.23
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.00
997,Christine,3.0,3.43
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.50


**Мы нашли количество обзоров и средний рейтинг для каждой книги, с помощью данного списка можно определить самые рейтинговые и популярные книги среди людей.** 

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [10]:
task = '''SELECT publisher,COUNT(*) as count
   FROM books as b 
   LEFT JOIN publishers as p ON b.publisher_id = p.publisher_id
        WHERE num_pages > 50
    GROUP BY publisher ORDER BY count DESC LIMIT 1
'''

con = engine.connect()
task = pd.io.sql.read_sql(sql = text(task), con = con)
print(f"Издательство, которые выпустила наибольшее число книг толще 50: {task['publisher'][0]}, количество книг {task['count'][0]}")



Издательство, которые выпустила наибольшее число книг толще 50: Penguin Books, количество книг 42


**Самым "активным", то есть выпускающим наибольшее количество, издельство среди книг является Penguin Books, в расчет мы не принимали брошюры. 
Достаточно интересный факт, надо также посмотреть и средний рейтинг, и популярность среди книголюбов. ВОзможно это хороший кандидат для включения в список издательств для нашего сервиса**

## Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
task = '''SELECT author, AVG(rating) as rating
   FROM books as b
RIGHT JOIN (SELECT book_id
          FROM ratings
    GROUP BY book_id 
    HAVING count(rating_id)>=50) as p ON b.book_id=p.book_id
LEFT JOIN authors a ON b.author_id = a.author_id
LEFT JOIN ratings r ON r.book_id = b.book_id
    GROUP BY author
    ORDER BY rating DESC LIMIT 1   
'''

con = engine.connect()
task = pd.io.sql.read_sql(sql = text(task), con = con)
print("Автор с самой высокой средней оценкой книг:", task['author'][0])

Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré


**Самый большой рейтинг у автора  J.K. Rowling/Mary GrandPré, также мы рассмотривали количество оценок равно или больше 50. Можно смело сказать, что данных автор популярен среди остальных и его книги стоит добавить в наш сервис**

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
task = '''
with a as (SELECT re.username , count(review_id) as review
 FROM reviews re
 RIGHT JOIN (SELECT username
     FROM ratings
    GROUP BY username HAVING COUNT(rating_id) > 48) as ra ON re.username = ra.username
 GROUP BY re.username)
 
select avg(review) from a
'''

con = engine.connect()
task = pd.io.sql.read_sql(sql = text(task), con = con)
print("Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:", task['avg'][0])
    

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.0


**Сайт для оценки книг достаточно популярен и пользователи часто оставляют там свои оценки. Количество пользователей, которые оставили более 48 оценок - 24**